Importando as bibliotecas


In [ ]:
#!pip install -U pandasql
import pandas as pd
import numpy as np
import pandasql

1 - Carregar o seu arquivo OVNIS.csv em um dataframe

In [ ]:
#Carregar o seu arquivo OVNIS.csv em um dataframe;
ovnis = pd.read_csv('OVNIS.csv')
ovnis

,Unnamed: 0,Date / Time,City,State,Shape,Duration,Summary,Posted
0,0,9/30/97 20:00,Nova Scotia (Canada),NS,Light,8-10 seconds.,"Single light resembling a star, but moving spu...",10/30/06
1,1,9/22/97 20:00,Solomons Island,MD,Disk,10 minutes,"Close up at twilight, Stationary UFO.",8/5/09
2,2,9/19/97,Garden Grove,CA,Rectangle,4 mins.,Around 6:30 PM I was walking through a Vons Pa...,12/1/19
3,3,9/18/97 20:15,Panama City,FL,Unknown,30 seconds,Looked like stars in the sky so far up/moveing...,3/13/12
4,4,9/15/97 00:00,Houston,TX,Disk,5 minutes,Beautiful silver-colored flying saucer about t...,7/19/10
...,...,...,...,...,...,...,...,...
71991,71991,12/1/16 18:00,Amherst,NY,Oval,5 minutes,"Odd, mostly stationary lights, in night sky, w...",12/5/16
71992,71992,12/1/16 17:30,Montgomery,AL,Circle,1 minute,2 extremely bright lights moving slowly and di...,12/5/16
71993,71993,12/1/16 16:30,Asbury Park,NJ,Oval,2 minutes,Jersey shore sighting.,12/5/16
71994,71994,12/1/16 12:46,Escondido,CA,Other,120 seconds,Reddish orange trail.,1/6/17


2 - Remover registros que tenham valores vazios (None, unknown, ...) para City, State e Shape;

In [ ]:
#Removendo os valores vazios e Unknown
ovnis.drop(ovnis.index[ovnis['City'] == 'None'], inplace = True)
ovnis.drop(ovnis.index[ovnis['City'] == None], inplace = True)
ovnis.drop(ovnis.index[ovnis['State'] == None], inplace = True)
ovnis.drop(ovnis.index[ovnis['Shape'] == None], inplace = True)
ovnis.drop(ovnis.index[ovnis['Shape'] == "Unknown"], inplace = True)
#Remove os valores NaN
ovnis['State'].dropna()
ovnis['Shape'].dropna()
ovnis['City'].dropna()

ovnis

,Unnamed: 0,Date / Time,City,State,Shape,Duration,Summary,Posted
0,0,9/30/97 20:00,Nova Scotia (Canada),NS,Light,8-10 seconds.,"Single light resembling a star, but moving spu...",10/30/06
1,1,9/22/97 20:00,Solomons Island,MD,Disk,10 minutes,"Close up at twilight, Stationary UFO.",8/5/09
2,2,9/19/97,Garden Grove,CA,Rectangle,4 mins.,Around 6:30 PM I was walking through a Vons Pa...,12/1/19
4,4,9/15/97 00:00,Houston,TX,Disk,5 minutes,Beautiful silver-colored flying saucer about t...,7/19/10
5,5,9/15/97 20:00,Santa Fe,NM,Light,2-3 minutes,Saw white dot of light moving in zig-zag motio...,11/9/17
...,...,...,...,...,...,...,...,...
71991,71991,12/1/16 18:00,Amherst,NY,Oval,5 minutes,"Odd, mostly stationary lights, in night sky, w...",12/5/16
71992,71992,12/1/16 17:30,Montgomery,AL,Circle,1 minute,2 extremely bright lights moving slowly and di...,12/5/16
71993,71993,12/1/16 16:30,Asbury Park,NJ,Oval,2 minutes,Jersey shore sighting.,12/5/16
71994,71994,12/1/16 12:46,Escondido,CA,Other,120 seconds,Reddish orange trail.,1/6/17


3 - Manter somente os registros referentes aos 51 estados dos Estados Unidos 

In [ ]:
#ovnis

q = """
  SELECT * FROM ovnis WHERE 
  STATE LIKE 'AL'
  OR STATE LIKE 'AK' 
  OR STATE LIKE 'AZ'
  OR STATE LIKE 'AR'
  OR STATE LIKE 'CA'
  OR STATE LIKE 'CO' 
  OR STATE LIKE 'CT' 
  OR STATE LIKE 'DE'
  OR STATE LIKE 'DC'
  OR STATE LIKE 'FL'
  OR STATE LIKE 'GA'
  OR STATE LIKE 'HI'
  OR STATE LIKE 'ID'
  OR STATE LIKE 'IL'
  OR STATE LIKE 'IN'
  OR STATE LIKE 'IA'
  OR STATE LIKE 'KS'
  OR STATE LIKE 'KY'
  OR STATE LIKE 'LA'
  OR STATE LIKE 'ME'
  OR STATE LIKE 'MT'
  OR STATE LIKE 'NE'
  OR STATE LIKE 'NV'
  OR STATE LIKE 'NH'
  OR STATE LIKE 'NJ'
  OR STATE LIKE 'NM'
  OR STATE LIKE 'NY'
  OR STATE LIKE 'NC'
  OR STATE LIKE 'ND'
  OR STATE LIKE 'OH'
  OR STATE LIKE 'OK'
  OR STATE LIKE 'OR'
  OR STATE LIKE 'MD'
  OR STATE LIKE 'MA'
  OR STATE LIKE 'MI'
  OR STATE LIKE 'MN'
  OR STATE LIKE 'MS'
  OR STATE LIKE 'MO'
  OR STATE LIKE 'PA'
  OR STATE LIKE 'RI'
  OR STATE LIKE 'SC'
  OR STATE LIKE 'SD'
  OR STATE LIKE 'TN'
  OR STATE LIKE 'TX'
  OR STATE LIKE 'UT'
  OR STATE LIKE 'VT'
  OR STATE LIKE 'VA'
  OR STATE LIKE 'WA'
  OR STATE LIKE 'WV'
  OR STATE LIKE 'WI'
  OR STATE LIKE 'WY' 
  
"""

eua_registros = pandasql.sqldf(q.lower(), locals())
eua_registros

,Unnamed: 0,Date / Time,City,State,Shape,Duration,Summary,Posted
0,1,9/22/97 20:00,Solomons Island,MD,Disk,10 minutes,"Close up at twilight, Stationary UFO.",8/5/09
1,2,9/19/97,Garden Grove,CA,Rectangle,4 mins.,Around 6:30 PM I was walking through a Vons Pa...,12/1/19
2,4,9/15/97 00:00,Houston,TX,Disk,5 minutes,Beautiful silver-colored flying saucer about t...,7/19/10
3,5,9/15/97 20:00,Santa Fe,NM,Light,2-3 minutes,Saw white dot of light moving in zig-zag motio...,11/9/17
4,6,9/15/97 20:00,Kent,WA,Sphere,5 minutes,Was looking thru a telescope at the moon in 19...,3/17/17
...,...,...,...,...,...,...,...,...
60341,71991,12/1/16 18:00,Amherst,NY,Oval,5 minutes,"Odd, mostly stationary lights, in night sky, w...",12/5/16
60342,71992,12/1/16 17:30,Montgomery,AL,Circle,1 minute,2 extremely bright lights moving slowly and di...,12/5/16
60343,71993,12/1/16 16:30,Asbury Park,NJ,Oval,2 minutes,Jersey shore sighting.,12/5/16
60344,71994,12/1/16 12:46,Escondido,CA,Other,120 seconds,Reddish orange trail.,1/6/17


4 - Remover variáveis irrelevantes para a análise (Duration, Summary e Posted);

In [ ]:
#eua_registros
#ajuste de colunas sem espaço
eua_registros.rename(columns = lambda x: x.replace(' ', '_'), inplace=True)

#Trocar o nome da coluna
eua_registros['DateTime'] = eua_registros['Date_/_Time']
#ovnis

#Filtro de dados
q = """
SELECT DateTime, City,	State,	Shape	
FROM eua_registros
"""

df1 = pandasql.sqldf(q, locals())
df1


,DateTime,City,State,Shape
0,9/22/97 20:00,Solomons Island,MD,Disk
1,9/19/97,Garden Grove,CA,Rectangle
2,9/15/97 00:00,Houston,TX,Disk
3,9/15/97 20:00,Santa Fe,NM,Light
4,9/15/97 20:00,Kent,WA,Sphere
...,...,...,...,...
60341,12/1/16 18:00,Amherst,NY,Oval
60342,12/1/16 17:30,Montgomery,AL,Circle
60343,12/1/16 16:30,Asbury Park,NJ,Oval
60344,12/1/16 12:46,Escondido,CA,Other


5 - Manter somente os registros de Shapes mais populares (com mais de 1000 ocorrências)

In [ ]:
n_shape = df1['Shape'].value_counts()
maioresque1000 = n_shape[n_shape > 1000]
maioresque1000

Light        13957
Circle        7467
Triangle      5953
Fireball      5915
Sphere        4560
Other         4116
Oval          2857
Disk          2806
Formation     2131
Changing      1603
Cigar         1308
Flash         1250
Rectangle     1135
Name: Shape, dtype: int64

In [ ]:

#Filtro de dados, agrupar depois filtrar
q = """
SELECT *
FROM df1
WHERE Shape in ('Light', 'Circle', 'Triangle', 'Fireball', 'Sphere', 'Other', 'Oval', 'Disk', 'Formation', 'Changing', 'Cigar', 'Flash', 'Rectangle')

"""


df2 = pandasql.sqldf(q, locals())
df2


,DateTime,City,State,Shape
0,9/22/97 20:00,Solomons Island,MD,Disk
1,9/19/97,Garden Grove,CA,Rectangle
2,9/15/97 00:00,Houston,TX,Disk
3,9/15/97 20:00,Santa Fe,NM,Light
4,9/15/97 20:00,Kent,WA,Sphere
...,...,...,...,...
55053,12/1/16 18:17,San Bernardino,CA,Light
55054,12/1/16 18:00,Amherst,NY,Oval
55055,12/1/16 17:30,Montgomery,AL,Circle
55056,12/1/16 16:30,Asbury Park,NJ,Oval


6 - Salvar o dataframe final

In [ ]:
#Gerar um arquivo csv
df2.to_csv('df_OVNI_limpo.csv')